# General  integrate-and-fire neuron

In gernal, the dynamics of a integrate-and-fire (IF) neuron can be described by 

\begin{equation}
\frac{dV}{dt}=f(V)+I(V,t)
\end{equation}

where f(V) is the deterministic drift towards the resting potential. E.g. for a 

-  Leaky IF neuron: &nbsp;$f(V)=-V/\tau$
-  Linear IF neuron: &nbsp; $f(V)=-\beta$ 

In the diffusion approximation, we assume that $I(V,t)$ can replaced by Gaussian white noise. Hence, we are arrive at a stochastic description and the dynamics are described by a Langevin equation  

\begin{equation}
\frac{dv}{dt}=h(v,t)+g(v,t)\Gamma(t)
\end{equation}

where $h(v,t)$ is sum of the deterministic drift plus the average afferent current $\mu(v,t)$ and $g(v,t)$ is the size of the current fluctuations. $\Gamma(t)$ is a Gaussian whites noise with zero mean and unitary variance. For a population, we  assume that the statistics of the afferent current at a given time are identical for all neurons. Furthermore, we assume that the noise feld by each neuron is statistically independent. This is called the mean-field assumption.<br> 
From the Langevin equation, we can derive a Fokker-Planck equation for density $\rho(v,t)$

\begin{equation}
\partial_{t}\rho(v,t)=L\rho(v,t)
\end{equation}

where $L$ is the Fokker-Planck operator 

\begin{equation}
L = -\partial_{v}h(v,t)+\frac{1}{2}\partial_{v}^{2}g(v,t)
\end{equation}

The Fokker-Planck equation can be written as a continuity equation 

\begin{equation}
\partial_{t}\rho=-\partial_{v}J,
\end{equation}

where we defined the probability flux 

\begin{equation}
J = (h(v,t)-\frac{1}{2}\partial_{v}g(v,t)) \rho(v,t).
\end{equation}

Spike generation can be incorporated into the model via boundary conditions. After, a neuron emits a spike, the membrane potential is clamped to reset for the refractory period $t_0$. If the probability mass reaches the threshold $v=\theta$ it is reinserted at the reset potential $v=H$. This behavior can be modeled by a absorbing boundary, i.e. $\rho(v \geq \theta, t) = 0$. Because the probability mass crossing the threshold is reinserted at reset, the flux makes a jump at $v=H$

\begin{equation}
J(\theta, t - t_0) = J(H_{+}, t) - J(H_{-}, t) 
\end{equation}

We may also define a lower bound $v_{\text{min}}$ (possibly $v_{\text{min}}=-\infty$) for the membrane potential which can be modelled by a reflecting boundary, i.e. $J(v_{\text{min}}, t) = 0$. To conclude, we have three different boundary conditions  

1. &nbsp; $\rho(v \geq \theta, t) = 0$
2. &nbsp; $J(\theta, t - t_0) = J(H_{+}, t) - J(H_{-}, t)$
3. &nbsp; $J(v \leq v_{\text{min}}, t) = 0$

The time-dependence of the Fokker-Planck operator is implicit through the time dependence of the input  statistics. For any specified input, the Fokker-Planck operator will have a set of eigenvectors $\phi_n$ with associated eigenvalues $\lambda_n$ such that 

\begin{equation}
L\phi_{n}=\lambda_{n}\phi_{n}
\end{equation}

$L$, $\phi_n$ and $\lambda_n$ all depend on the input statistics, and therefore on time 

\begin{equation}
L=L(t),\,\phi_{n}=\phi_{n}(t),\,\lambda_{n}=\lambda_{n}(t)
\end{equation}

Note that the eigenfunctions $\phi_{n}$ are not orthogonal, because $L$ is not Hermitian. We define the inner product

\begin{equation}
\bigl\langle\psi,\phi\bigr\rangle=\int dv\,\psi(v)\phi(v)
\end{equation}

adjoint operator $L^{\dagger}$ such that  

\begin{equation}
\bigl\langle\psi,L\phi\bigr\rangle=\bigl\langle L^{\dagger}\psi,\phi\bigr\rangle
\end{equation}

where $\phi$ satisfies the previously defined boundary conditions. Using the above definition, we find the adjoint operator

\begin{equation}
L^{\dagger}=h(v,t)\partial_{v}+\frac{1}{2}g(v,t)\partial_{v}^{2},
\end{equation}

and the boundary conditions for the adjoint eigenfunctions $\psi_n$

1. &nbsp; $\psi_{n}(\theta,t)S_{\phi_{n}}(\theta,t)=\psi_{n}(H,t)S_{\phi_{n}}(\theta,t-\tau_{0})$  
2. &nbsp; $\partial_{v}\psi_{n}(v_{\text{min}},t)=0$
3. &nbsp; $\partial_{v}\psi_{n}(H_{+},t)=\partial_{v}\psi_{n}(H_{-},t)$

## Eigenfunction expansion

The pairs of eigenfunctions $(\psi_{n}, \phi_{n})$ span a complete bi-orthogonal basis  

\begin{equation}
\sum_{n}\phi_{n}(v)\psi_{n}(v')=\delta(v-v')
\end{equation}

so that we can expand the probability density in terms of eigenfunctions 

\begin{equation}
\rho(v)  =\sum_{n} a_{n} \phi_{n}(v),
\end{equation}

where the coefficients $a_{n}$ are given by  

\begin{equation}
a_{n}=\int dv\,\psi_{n}(v)\rho(v)=\bigl\langle\psi_{n},\,\rho\bigr\rangle
\end{equation}

Using the above equation, the dynamics of the coefficients are described by the   

\begin{equation}
\partial_t a = \partial_t \bigl\langle\psi_{n},\,\rho\bigr\rangle =\bigl\langle\psi_{n}, \partial_{t} \rho \bigr\rangle+ \bigl\langle\ \partial_{t} \psi_n, \rho\bigr\rangle 
\end{equation}

Substituting $\partial_{t} \rho = L\rho$ and expression $\rho$ in terms of the eigenfunctions $\phi_n$ yields   

\begin{equation}
\dot{a}_{n}=	\sum_{m}\left(\lambda_{n}a_{m}\bigl\langle\psi_{n},\phi_{m}\bigr\rangle+a_{m}\bigl\langle\partial_{t}\psi_{n},\phi_{m}\bigr\rangle\right)=\lambda_{n}a_{n}+\sum_{m}a_{m}\bigl\langle\partial_{t}\psi_{n},\phi_{m}\bigr\rangle
\end{equation}

Coupling term contains the derivatives $\partial_{t}\psi_{n}$. Note that $\psi_{n}$ depends on time due to the time dependence of the Fokker-Planck operator. The time dependence of the Fokker-Planck operator is implicit through the rate $r(t)$ which enters into the drift and diffusion coefficient so that  

\begin{equation}
\dot{a}_{n}=\lambda_{n}a_{n}+ \dot{r} \sum_{m}a_{m}\bigl\langle \partial_{r}\psi_{n},\phi_{m}\bigr\rangle
\end{equation}

We distinguish between the internal rate $r_{\text{int}}(t)$ a neuron feels due recurrent connections from other neurons in the population and the external rate $r_{\text{ext}}(t)$ due to connections from neurons in the external bath surrounding the population so that 

\begin{equation}
r(t)=r_{\text{int}}(t)+r_{\text{ext}}(t).
\end{equation}

The average interal rate at a given time is equivalent to the probality flux crossing threshold $r_{\text{int}}(t)=J(\theta,t - t_{0})$. Inserting the definition of $J$ and using that $\rho(\theta, t)$ vanishes at the boundary, we arrive at 

\begin{equation}
r_{\text{int}}(t)=J(\theta,t-t_{0})=\left.-\frac{1}{2}\partial_{v}g(v,t-t_{0}))\rho(v,t-t_{0})\right|_{v=\theta}
\end{equation}

Expanding $\rho(v,t-t_{0})$ in terms of eigenfunctions yields 

\begin{equation}
r_{\text{int}}(t)=\left.-\frac{1}{2}\sum_{n}a_{n}(t-t_{0})\partial_{v}g(v,t-t_{0})\phi_{n}(v,t-t_{0})\right|_{v=\theta}
\end{equation}

It is convenient to introduce a notation which separates the zero (steady-state) mode from the nonstationary higher modes. We define  

\begin{equation}
r_{\text{int}}^{(0)}(t) =\left.-\frac{1}{2}\partial_{v}g(v,t-t_{0})\phi_{0}(v,t-t_{0})\right|_{v=\theta}
\end{equation}

where we used $a_{0}(t) = 1$. In the same spirit, we can write

\begin{equation}
\dot{a}_{n} = \lambda_{n}a_{n}+\dot{r} a_{0}\bigl\langle\partial_{r}\psi_{n},\phi_{0}\bigr\rangle+\dot{r}\sum_{m=1}a_{m}\bigl\langle\partial_{r}\psi_{n},\phi_{m}\bigr\rangle
\end{equation}

Defining the coefficient vector $\boldsymbol{a}=(a_{1},a_{2}, \ldots)$, the above equation can be written in matrix notation 

\begin{equation}
\dot{\boldsymbol{a}} =\left(\boldsymbol{\Lambda}+\boldsymbol{C}\dot{r}\right)\boldsymbol{a}+\boldsymbol{c}\dot{r}
\end{equation}

where $\boldsymbol{\Lambda}$ is a diagonal matrix with 

\begin{equation}
\Lambda_{nm}=\lambda_{n}\delta_{nm},
\end{equation}

$\boldsymbol{C}$ the coupling matrix between nonstationary modes 

\begin{equation}
C_{nm}=\bigl\langle\partial_{r}\psi_{n}, \phi_{m}\bigr\rangle,
\end{equation}

and the elements of vector $\boldsymbol{c}$ are the coupling terms between the nonstationary modes 

\begin{equation}
c_{n}=\left\langle \partial_{r}\psi_{n},\phi_{0}\right\rangle.
\end{equation}

The rate equation can be written as 

\begin{equation}
r=r_{\text{int}}^{(0)}+r_{\text{int}}^{(n>0)}+r_{\text{ext}}
\end{equation}

where $r_{\text{int}}^{(n>0)}$ is interal rate contribution due to nonstationary modes. It can be written as a scalar product  

\begin{equation}
r_{\text{int}}^{(n>0)} = \boldsymbol{f}\boldsymbol{a}
\end{equation}

where we defined the vector $\boldsymbol{f}$ whose elements are the flux over boundary of nonstationary modes 

\begin{equation}
f_{n}=\left. -\frac{1}{2} g(v,t-t_{0})\partial_{v}\phi_{n}(v,t-t_{0})\right|_{v=\phi}.
\end{equation}

Assuming that a finite and small number of higher modes is sufficient to describe the time evolution of $r$. This ansatz provides effective way to reduce the dimensionality of the problem. To use the method, we need to know the eigenvalues and eigenfunctions of the Fokker-Planck operator in dependence of the rate.  

# Linear IF neuron

Apply method to a linear integrate-and-fire neuron 

\begin{equation}
\frac{dV}{dt}=-\beta+I(V,t).
\end{equation}

In the following, we assume that the average afferent current $\mu(v,t)$ and the size of the current fluctuations $g(v,t)$ do not depend on the membrane potential. Hence, the Fokker-Planck operator at time $t$ is given by 

\begin{equation}
L =-h(t)\partial_{v}+\frac{1}{2}g(t)\partial_{v}^{2}.
\end{equation}

where the drift coefficient $h(t)$ is given $h(t)=-\beta + \mu(t)$. 

## Eigenfunctions

The eigenfunctions $\phi_n(v)$ fulfill $L\phi_{n}=\lambda_{n}\phi_{n}$ which leads to a second order ODE with constant coefficients   

\begin{equation}
-h\partial_{v}\phi_{n}(v)+\frac{1}{2}g\partial_{v}^{2}\phi_{n}(v)=\lambda_{n}\phi_{n}(v)
\end{equation}

where we supressed the time arguments. The ansatz $\phi_{n}(v)=e^{\alpha_{n} v}$ yields the general solution

\begin{equation}
\phi_{n}(v)=c_{1} e^{\alpha_{1}v}+c_{2}e^{\alpha_{2}v}
\end{equation}

where $\alpha_{1}$ and $\alpha_{2}$ are given by 

\begin{equation}
\alpha_{1/2} = \pm\frac{1}{g}\sqrt{2\lambda_{n} g+h^{2}}+\frac{h}{g}
\end{equation}

We define $\zeta=\frac{\theta}{g}\sqrt{2\lambda_{n} g+ h^{2}}$ and $\xi=\frac{\theta h}{g}$ so that 

\begin{equation}
\alpha_{1/2} = \frac{\pm \zeta + \xi}{\theta}
\end{equation}

Hence, we find 

\begin{equation}
\phi_{n}(v)= \left(c_{1}e^{\frac{\zeta}{\theta}v}+c_{2}e^{\frac{-\zeta}{\theta}v}\right)e^{\frac{\xi}{\theta}v}
\end{equation}

The coefficients $c_{1}$ and $c_{2}$ needs to be determined from the boundary conditions. From $\phi(\theta,t)=0$ follows 

\begin{equation}
\phi_{n}(v) = c_n \sinh\left(\zeta_n\frac{\theta_n - v}{\theta}\right)e^{\frac{\xi v}{\theta}}
\end{equation}

Note that $\zeta_n$ depends on the eigenvalue $\lambda_n$. Using the same ansatz and the boundary conditions for the adjoint eigenfunctions, we find 

\begin{equation}
\psi_{n}(v,t)=\left[\xi\cosh\frac{\zeta_n v}{\theta}+\xi\sinh\frac{\zeta_n v}{\theta}\right]e^{-\frac{\xi}{\theta}v}
\end{equation}

Here, we absorbed the undetermined coefficient into the coefficient $c_n$ in the expression for $\phi_n$. The  coefficient $c_n$ must be determined by the orthonormality constraint $\left\langle \psi_{n},\phi_{m}\right\rangle =\delta_{nm}$. 

\begin{equation}
c_n= \frac{2\zeta_n}{\theta\left[\zeta_n \xi\cosh\left(\zeta\right)+(\zeta_n^{2}-\xi)\sinh\zeta_n\right]}
\end{equation}

The boundary condition which ensures flux conservation over the reflecting barrier 

\begin{equation}
\left.\frac{1}{2}g \partial_{v}\phi_{n}\right|_{v=\theta}=\left.\left(\frac{1}{2}g\partial_{v}\phi_{n}-\mu\phi_{n}\right)\right|_{v\rightarrow0^{+}}
\end{equation}

yields the characteristic equation determining the eigenvalues $\lambda_n$ of the Fokker-Planck operator 

\begin{equation}
\zeta e^{\xi}=\zeta\cosh\left(\zeta\right)+\xi\sinh\left(\zeta\right)
\end{equation}
 
 
 

In [ ]:
import numpy as np

# parameter 
h = 0.01 # efficacy
theta = 1 # threshold 
tau = 10*1e-3 

r = 5.0 # spontaneous activity 

sig2 = tau*
# rates need to be determined by self-consistency equation 

    sig2/2mu**2*(2*mu*theta/sig2**2 - 1 + np.exp(-2*mu*theta/sig**2))


def transduction_function():
    
    sig2 = 0 
    mu = 0 
    

    
    
    




# Leaky IF neuron


For a leaky IF neuron the Fokker-Planck operator is given by 

\begin{equation}
L=\partial_{v}\frac{1}{\tau}(v-\mu)+\frac{\sigma^{2}}{\tau}\partial_{v}^{2} = \frac{1}{\tau}+\frac{1}{\tau}(v-\mu)\partial_{v}+\frac{\sigma^{2}}{\tau}\partial_{v}^{2}
\end{equation}

where $\sigma^{2}=\frac{\tau h^{2}}{2}r$ and $\mu=\tau hr$. Hence, the eigenfunction equation 

\begin{equation}
\frac{1}{\tau}\phi_{n}+\frac{1}{\tau}(v-\mu)\partial_{v}\phi_{n}+\frac{\sigma^{2}}{\tau}\partial_{v}^{2}\phi_{n}=\lambda_{n}\phi_{n}
\end{equation}

is a second-order homogeneous ordinary differential equation with variable coefficients. The above equation can written as 

\begin{equation}
\phi''_{n}+a(v)\phi'_{n}+b\phi_{n}=0
\end{equation}

where we defined 

\begin{equation}
a(v)=\frac{v-\mu}{\sigma^{2}},\quad b=\frac{1-\tau\lambda_{n}}{\sigma^{2}}
\end{equation}

How can we solve such a equation? 

- Method of reduction (requires us to know one solution)


# Bibliography 

1.  Mattia, M., & Del Giudice, P. (2002). Population dynamics of interacting spiking neurons. Physical Review E - Statistical, Nonlinear, and Soft Matter Physics, 66(5), 1–19.